In [1]:
#%pip install uproot awkward tqdm mplhep plotly pandas fastjet

In [2]:
from utils import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import uproot
import awkward as ak

In [3]:
data = uproot.open("data/l1Nano.root")["Events"].arrays(
    [ f"{p}_{v}" for p in ("Puppi","PF","PuppiExtended") for v in ("pt","eta","phi","pdgId","z0","dxy","charge")] + 
    [ f"PuppiJet_{v}" for v in ("pt","eta","phi","mass","numberOfDaughters")] + [ "PuppiExtended_sc4JetIdx" ] +
    [ f"{j}_{v}" for j in ("AK4L1PuppiJet","AK4L1PFJet","AK4L1PuppiExtJet") for v in ("pt","eta","phi","mass","numberOfDaughters")] + 
    [ f"{p}_ak4JetIdx" for p in ("Puppi","PF","PuppiExtended")] + 
    [ f"GenJet_{v}" for v in ("pt","eta","phi","mass") ],
    library="ak"
)

print("Loaded", len(data["Puppi_pt"]), "events.")

Loaded 100 events.


## Run clustering

In [5]:
EVENT = 0
candLabel = "PuppiExtended"
jetLabels = [ "AK4L1PuppiExtJet", "PuppiJet" ]

print(f"Running clustering on event {EVENT}...")

cands = tuple(data[f"{candLabel}_{v}"][EVENT] for v in ("pt","eta","phi"))
jets  = dict()
for jl in jetLabels:
    jets[jl] = tuple(data[f"{jl}_{v}"][EVENT] for v in ("pt","eta","phi"))

from collections import defaultdict
ak4rev = defaultdict(list)
sc4rev = defaultdict(list)
for i,(pt,eta,phi) in enumerate(zip(*cands)): # sort by eta
    ak4i = data[f"{candLabel}_ak4JetIdx"][EVENT][i]
    ak4rev[ak4i].append(i)
    extra = ""
    if candLabel == "PuppiExtended":
        sc4i = data[f"{candLabel}_sc4JetIdx"][EVENT][i]
        sc4rev[sc4i].append(i)
        extra = f"  sc4i {sc4i:3d}"       
    print(f"Cand {i:3d}  pt {pt:7.2f}  eta {eta:+5.2f}  phi {phi:+5.2f}   ak4i {ak4i:3d}{extra}")
print("")

for jl in jetLabels:
    for i,(pt,eta,phi) in sorted(enumerate(zip(*jets[jl])), key = lambda x : -x[1][0]): # sort by pt, keep original index
        if "AK4" in jl:
            extra = "  cands: " + ", ".join(map(str, sorted(ak4rev[i])))
        elif jl == "PuppiJet":
            extra = "  cands: " + ", ".join(map(str, sorted(sc4rev[i])))            
        print(f"{jl} {i:3d}  pt {pt:7.2f}  eta {eta:+5.2f}  phi {phi:+5.2f}{extra}")
    print("")
    if "AK4" in jl:
        (cpt, ceta, cphi) = cands
        pyjets, assigns = antikt_jets(ceta, cphi, cpt, R_clu=0.4, pt_min=5.0)
        pyrev = dict((i,[]) for i in range(-1, len(pyjets[0])))
        for c, j in enumerate(assigns):
            pyrev[j].append(c)
        for i,(pt,eta,phi,m) in sorted(enumerate(zip(*pyjets)), key = lambda x : -x[1][0]): # sort by pt, keep original index
            print(f"PyAK4 {i:3d}  pt {pt:7.2f}  eta {eta:+5.2f}  phi {phi:+5.2f}  cands: " + ", ".join(map(str, sorted(pyrev[i]))))
        print("")
    if jl == "PuppiJet":
        (cpt, ceta, cphi) = cands
        pyjets, assigns = seeded_cone_jets_iter(ceta, cphi, cpt, R_clu=0.4, nseeds=16)
        pyrev = dict((i,[]) for i in range(-1, len(pyjets[0])))    
        for c, j in enumerate(assigns):
            pyrev[j].append(c)
        for i,(pt,eta,phi,m) in sorted(enumerate(zip(*pyjets)), key = lambda x : -x[1][0]): # sort by pt, keep original index
            print(f"PySC4 {i:3d}  pt {pt:7.2f}  eta {eta:+5.2f}  phi {phi:+5.2f}  cands: " + ", ".join(map(str, sorted(pyrev[i]))))        
            
        

Running clustering on event 0...
Cand   0  pt   25.75  eta +2.19  phi +0.00   ak4i   0  sc4i   3
Cand   1  pt    3.50  eta +2.39  phi -0.34   ak4i   0  sc4i   3
Cand   2  pt   21.75  eta +2.18  phi +0.02   ak4i   0  sc4i   3
Cand   3  pt    2.50  eta +1.97  phi +0.37   ak4i  -1  sc4i  11
Cand   4  pt   15.75  eta +2.21  phi -0.02   ak4i   0  sc4i   3
Cand   5  pt    2.25  eta +1.92  phi -0.41   ak4i  -1  sc4i   9
Cand   6  pt    3.75  eta -1.08  phi +0.59   ak4i  -1  sc4i   7
Cand   7  pt   10.75  eta +2.16  phi +0.03   ak4i   0  sc4i   3
Cand   8  pt    2.25  eta +1.94  phi -0.29   ak4i   0  sc4i   3
Cand   9  pt    5.75  eta -0.99  phi -1.54   ak4i   5  sc4i   5
Cand  10  pt    5.75  eta +2.23  phi +0.03   ak4i   0  sc4i   3
Cand  11  pt    2.00  eta +2.18  phi +0.14   ak4i   0  sc4i   3
Cand  12  pt    3.75  eta +2.22  phi +0.02   ak4i   0  sc4i   3
Cand  13  pt    1.00  eta -0.85  phi -1.56   ak4i   5  sc4i   5
Cand  14  pt    3.25  eta -2.22  phi +1.95   ak4i  -1  sc4i   8
Cand  1